In [ ]:
import numpy as np
from copy import deepcopy
from math import exp, sqrt
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [20]:
def gaussian_mixture_field(size, n_components=None):
    x, y = np.meshgrid(np.linspace(0, 1, size), np.linspace(0, 1, size))
    field = np.zeros((size, size))

    if not n_components:
        n_components = np.random.poisson(20)

    for _ in range(n_components):
        cx, cy = np.random.uniform(0, 1, 2) # random center
        sx, sy = np.random.uniform(0.01, 0.2, 2) # random covariance scale
        w = np.random.exponential(10) # weight scale

        gaussian = w * np.exp(-(((x - cx) ** 2) / (2 * sx**2) +
                                ((y - cy) ** 2) / (2 * sy**2)))
        field += gaussian

    field = (1 - (field - field.min()) / (field.max() - field.min()))*2
    return field

In [21]:
class Simulator():
    def __init__(self, size=256, wind_speed=0, wind_direction=[0,0], response_rate=0.1, response_start=20, base_spread_rate=0.3, n_components=None, decay_rate=1e-3):
        self.size = size
        self.map = np.zeros((size, size))
        self.wind_speed = wind_speed
        self.wind_direction = wind_direction
        self.response_rate = response_rate
        self.response_start = response_start
        self.spread_rate = base_spread_rate
        self.time = 0
        self.decay_rate = decay_rate
        self.maps = {}

        self.terrain = gaussian_mixture_field(size, n_components=n_components)

    def step(self):
        new_map = deepcopy(self.map)

        for i in range(self.size):
            for j in range(self.size):
                if self.map[i, j] >= 1:
                    if np.random.rand() < self.spread_rate*self.terrain[i, j]*np.exp(-self.decay_rate * self.time) and new_map[i, j] < 5:
                        new_map[i, j] += 1
                    
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue

                            ni, nj = i + di, j + dj
                            spread_chance = self.spread_rate*self.map[i,j]

                            if 0 <= ni < self.size and 0 <= nj < self.size:
                                if self.wind_speed > 0:
                                    wind_influence = (di * self.wind_direction[0] + dj * self.wind_direction[1]) / (np.linalg.norm(self.wind_direction) + 1e-6)
                                    wind_influence *= np.random.normal(1, 0.5)


                                    if wind_influence > 0:
                                        spread_chance *= (1 + self.wind_speed * wind_influence)
                                    spread_chance *= self.terrain[ni, nj]
                                    spread_chance *= np.exp(-self.decay_rate * self.time)
                                    spread_chance = np.clip(spread_chance, 0, 1)

                                if np.random.rand() < spread_chance and new_map[ni, nj] <= new_map[i, j]:
                                    if new_map[ni, nj] < 5:
                                        new_map[ni, nj] += 1

                                if self.time >= self.response_start and new_map[ni, nj] == 0:
                                    if np.random.rand() < 1 - exp(-(self.response_rate*(0.5+self.terrain[i,j]) * (self.time - self.response_start))):
                                        if new_map[i, j] > 0:
                                            new_map[i, j] -= 1 # Firefighting effort
                            if np.exp(-self.decay_rate * self.time) < 0.5:
                                if ni < 0 or ni >= self.size or nj < 0 or nj >= self.size:
                                    if np.random.rand() < 1 - exp(-(self.response_rate) * (self.time - self.response_start)):
                                        if new_map[i, j] > 0:
                                            new_map[i, j] -= 1 # Edge effect
                    
                else:
                    if 1 < i < self.size - 1 and 1 < j < self.size - 1:
                        neighbors_on_fire = np.sum(self.map[i-1:i+2, j-1:j+2] >= 1) - (1 if self.map[i, j] >= 1 else 0)
                        if neighbors_on_fire >= 6 and new_map[i, j] == 0:
                            new_map[i, j] += 1
        
        self.maps[self.time] = deepcopy(self.map)
        self.map = new_map
        self.time += 1

    def simulate(self):
        nodes = np.random.poisson(3)

        x_init, y_init = np.random.randint(0, self.size, size=2)
        self.map[x_init, y_init] = np.random.poisson(3)

        for _ in range(nodes - 1):
            while True:
                x, y = np.random.randint(-20, 21, size=2)
                if x_init+x < self.size and y_init+y < self.size:
                    break
                
            if self.map[x_init+x, y_init+y] == 0:
                self.map[x_init+x, y_init+y] = np.random.poisson(3)
                break
    
        while np.any(self.map > 0):
            self.step()

In [ ]:
simulator = Simulator(size=256, wind_speed=1.5, wind_direction=[1,2], response_rate=0.05, response_start=100, base_spread_rate=0.05)
simulator.simulate()

In [ ]:
def make_heatmap_gif(simulator, filename="simulation.gif", cmap="plasma"):
    times = sorted(simulator.maps.keys())
    frames = [simulator.maps[t] for t in times]

    fig, ax = plt.subplots()

    # --- terrain as background ---
    terrain_img = ax.imshow(simulator.terrain, cmap="Greens", alpha=0.4)

    # --- fire layer on top ---
    vmin, vmax = np.min(frames), np.max(frames)
    fire_img = ax.imshow(frames[0], cmap=cmap, vmin=vmin, vmax=vmax)

    cbar = fig.colorbar(fire_img, ax=ax)
    cbar.set_label("Fire Intensity", rotation=270, labelpad=15)

    def update(frame):
        fire_img.set_data(frame)   # only update the fire layer
        return [fire_img]

    ani = animation.FuncAnimation(
        fig, update, frames=frames, interval=60, blit=True
    )

    ani.save(filename, writer="pillow")
    plt.close(fig)


make_heatmap_gif(simulator, "fire.gif")